# OpenBB Data Base

In [1]:
from openbb import obb
import pandas as pd

In [2]:
# from dotenv import load_dotenv
import os

key = os.getenv("PAT_OBB")
obb.account.login(pat=key)

## Download Company Ticker Data

In [2]:
all_companies = obb.equity.search("", provider="sec")
len(all_companies.results)

9708

In [3]:
sec_df = all_companies.to_df()

In [4]:
sec_df.head()

,symbol,name,cik
0,AAPL,Apple Inc.,320193
1,MSFT,MICROSOFT CORP,789019
2,NVDA,NVIDIA CORP,1045810
3,AMZN,AMAZON COM INC,1018724
4,GOOGL,Alphabet Inc.,1652044


In [30]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"


tables = pd.read_html(url)
sp500_table = tables[0]
sp500_tickers = sp500_table['Symbol'].tolist()

len(sp500_tickers)

503

In [49]:
stocks = sec_df['symbol'].tolist()
stocks = stocks[350:600]

In [20]:
eu600 = pd.read_csv('data/eurostoxx600.csv' , sep=';')

eu600.head()

,Creation_Date,Index_Symbol,Index_Name,Index ISIN,Internal_Key,ISIN,RIC,Instrument_Name,Country,Currency,Exchange,Index Membership,Rank (FINAL),Rank (PREVIOUS),Comment,Rank 2 (FINAL),Rank 2 (PREVIOUS)
0,20250303,SXXP,STOXX Europe 600,EU0009658202,465191,DK0062498333,NOVOb.CO,NOVO NORDISK B,DK,DKK,NASDAQ Copenhagen,NaN,1.0,3.0,NaN,NaN,NaN
1,20250303,SXXP,STOXX Europe 600,EU0009658202,546078,NL0010273215,ASML.AS,ASML HLDG,NL,EUR,Euronext Amsterdam,NaN,2.0,1.0,NaN,NaN,NaN
2,20250303,SXXP,STOXX Europe 600,EU0009658202,476361,DE0007164600,SAPG.DE,SAP,DE,EUR,XETRA,NaN,3.0,2.0,NaN,NaN,NaN
3,20250303,SXXP,STOXX Europe 600,EU0009658202,461669,CH0038863350,NESN.S,NESTLE,CH,CHF,Six Swiss Exchange,NaN,4.0,4.0,NaN,NaN,NaN
4,20250303,SXXP,STOXX Europe 600,EU0009658202,098952,GB0009895292,AZN.L,ASTRAZENECA,GB,GBP,London SE,NaN,5.0,6.0,NaN,NaN,NaN


In [21]:
eu600 = eu600[eu600["Rank (FINAL)"] <= 600]

In [22]:
eu600.loc[:, 'symbol'] = eu600['RIC'].str.split('.').str[0]

eu600.head()

,Creation_Date,Index_Symbol,Index_Name,Index ISIN,Internal_Key,ISIN,RIC,Instrument_Name,Country,Currency,Exchange,Index Membership,Rank (FINAL),Rank (PREVIOUS),Comment,Rank 2 (FINAL),Rank 2 (PREVIOUS),symbol
0,20250303,SXXP,STOXX Europe 600,EU0009658202,465191,DK0062498333,NOVOb.CO,NOVO NORDISK B,DK,DKK,NASDAQ Copenhagen,NaN,1.0,3.0,NaN,NaN,NaN,NOVOb
1,20250303,SXXP,STOXX Europe 600,EU0009658202,546078,NL0010273215,ASML.AS,ASML HLDG,NL,EUR,Euronext Amsterdam,NaN,2.0,1.0,NaN,NaN,NaN,ASML
2,20250303,SXXP,STOXX Europe 600,EU0009658202,476361,DE0007164600,SAPG.DE,SAP,DE,EUR,XETRA,NaN,3.0,2.0,NaN,NaN,NaN,SAPG
3,20250303,SXXP,STOXX Europe 600,EU0009658202,461669,CH0038863350,NESN.S,NESTLE,CH,CHF,Six Swiss Exchange,NaN,4.0,4.0,NaN,NaN,NaN,NESN
4,20250303,SXXP,STOXX Europe 600,EU0009658202,098952,GB0009895292,AZN.L,ASTRAZENECA,GB,GBP,London SE,NaN,5.0,6.0,NaN,NaN,NaN,AZN


In [23]:
eu600_symbols = eu600['symbol'].tolist()

In [24]:
len(eu600_symbols)

600

In [ ]:
obb.index.constituents(symbol="nasdaq", provider="fmp")

## Download Ratio Data

In [50]:
from pandas.errors import EmptyDataError

def download_ratios(stocks, provider="fmp", limit=10):# limit=10 for 10 year historical data, however, the free plan only allows 5 years

    ratio_list = []

    for stock in stocks:
        try:
            df = obb.equity.fundamental.ratios(stock, provider=provider, limit=limit).to_df() 
            df['Stock'] = stock
            ratio_list.append(df)

        except EmptyDataError:
            print(f"No data found for the symbol {stock}")

        except Exception as e:
            print(f"Error with {stock}: {e}")


    return pd.concat(ratio_list, ignore_index=True) if ratio_list else pd.DataFrame()


Error with BRK.B: 
[Empty] -> No data found for the symbol BRK.B.
Error with BF.B: 
[Empty] -> No data found for the symbol BF.B.
Error with PODD: 
[Error] -> Unauthorized FMP request -> Limit Reach . Please upgrade your plan or visit our documentation for more details at https://site.financialmodelingprep.com/


In [ ]:
df_ratios = download_ratios(stocks) # stocks = list of stock symbols to download ratios for
# df_ratios.to_csv('data/namecsv.csv', index=False), run it the first time to save the data to a csv file

In [51]:
ratio_df = pd.read_csv('data/namecsv.csv')
df_ratios = df_ratios[ratio_df.columns]

,period_ending,fiscal_period,fiscal_year,current_ratio,quick_ratio,cash_ratio,days_of_sales_outstanding,days_of_inventory_outstanding,operating_cycle,days_of_payables_outstanding,...,price_to_free_cash_flows_ratio,price_to_operating_cash_flows_ratio,price_cash_flow_ratio,price_earnings_to_growth_ratio,price_sales_ratio,dividend_yield,enterprise_value_multiple,price_fair_value,interest_coverage,Stock
0,2024-12-31,FY,2024,1.411158,1.082623,0.497512,48.597355,93.429086,142.026441,67.204264,...,111.446351,39.088935,39.088935,-0.106482,2.893297,0.027875,16.416792,18.506708,NaN,MMM
1,2023-12-31,FY,2023,1.070733,0.755508,0.387854,53.050702,95.255182,148.305884,64.102668,...,9.995352,7.578811,7.578811,0.032353,1.549110,0.065401,-9.122197,10.531820,-9.690021,MMM
2,2022-12-31,FY,2022,1.542371,0.978263,0.383808,48.326857,101.954035,150.280892,60.409474,...,14.770213,10.149734,10.149734,-50.244370,1.657868,0.059369,7.945437,3.842056,14.153680,MMM
3,2021-12-31,FY,2021,1.704815,1.153071,0.505147,48.109178,96.808992,144.918170,58.143655,...,14.696170,11.535724,11.535724,1.711831,2.432111,0.039773,10.343493,5.688119,11.176230,MMM
4,2020-12-31,FY,2020,1.885003,1.351661,0.583040,53.359589,93.178862,146.538450,56.294188,...,12.766253,10.404347,10.404347,0.812508,2.622746,0.040137,10.673665,6.527760,8.453686,MMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,2024-12-31,FY,2024,2.289021,1.709000,0.847325,67.369869,94.729397,162.099266,75.747601,...,29.249410,26.126988,26.126988,5.221784,5.043755,0.000885,19.121151,3.584985,6.098030,IR
1230,2023-12-31,FY,2023,2.216604,1.668746,0.873146,65.514318,91.489897,157.004215,73.221162,...,24.612604,22.729223,22.729223,1.435874,4.553051,0.001035,19.720039,3.199905,7.430121,IR
1231,2022-12-31,FY,2022,2.369952,1.757407,0.963560,69.220628,104.233436,173.454064,79.156014,...,27.656948,24.615745,24.615745,1.108984,3.579420,0.001530,17.006954,2.302891,7.919574,IR
1232,2021-12-31,FY,2021,2.803638,2.221639,1.437351,67.199557,98.543886,165.743443,77.351528,...,46.542757,41.695656,41.695656,-0.031519,4.980917,0.000320,25.844592,2.851044,6.450399,IR


In [ ]:
df_ratios = df_ratios[~df_ratios['Stock'].isin(ratio_df['Stock'])]

df_ratios.head()

In [ ]:
print(len(ratio_df['Stock'].unique()) + len(df_ratios['Stock'].unique()))

In [ ]:
updated_df = pd.concat([ratio_df, df_ratios], ignore_index=True)

updated_df.tail()

In [58]:
len(updated_df['Stock'].unique())

867

In [59]:
updated_df.shape

(4328, 58)

In [60]:
updated_df.to_csv("data/namecsv.csv", index=False)

## Download Metrics Data

In [3]:
stocks = pd.read_csv("data/stocks.csv")
companies = stocks['Stock'].unique().tolist()

In [4]:
metrics_df = pd.read_csv('data/stocks_metrics.csv')
downloaded_companies = metrics_df['Stock'].unique().tolist()

In [7]:
def get_remaining_companies(companies, downloaded_companies, limit=250):
    return [company for company in companies if company not in downloaded_companies][:limit]

In [8]:
companies = get_remaining_companies(companies, downloaded_companies)

In [9]:
companies[:5]

['BN', 'BNPQY', 'BNS', 'BNTX', 'BP']

In [11]:
from pandas.errors import EmptyDataError

def download_metrics(stocks, provider="fmp", limit=10):# limit=10 for 10 year historical data, however, the free plan only allows 5 years

    metric_list = []

    for stock in stocks:
        try:
            df = obb.equity.fundamental.metrics(stock,provider=provider, limit=limit).to_df() 
            df['Stock'] = stock
            metric_list.append(df)

        except EmptyDataError:
            print(f"No data found for the symbol {stock}")

        except Exception as e:
            print(f"Error with {stock}: {e}")


    return pd.concat(metric_list, ignore_index=True) if metric_list else pd.DataFrame()

In [12]:
df_metrics = download_metrics(companies) 

In [13]:
df_metrics = df_metrics[metrics_df.columns]

In [14]:
df_metrics = df_metrics[~df_metrics['Stock'].isin(metrics_df['Stock'])]

df_metrics.head()

,symbol,market_cap,pe_ratio,period_ending,fiscal_period,calendar_year,revenue_per_share,capex_per_share,net_income_per_share,operating_cash_flow_per_share,...,receivables_turnover,payables_turnover,inventory_turnover,return_on_equity,return_on_invested_capital,return_on_tangible_assets,dividend_yield,graham_number,graham_net_net,Stock
0,BN,6.034211e+10,94.137458,2024-12-31,FY,2024,79.082539,10.636542,0.610278,7.206229,...,4.095235,2.149261,11.950257,0.013950,-0.288754,0.001532,0.010987,24.509301,-273.000210,BN
1,BN,6.252702e+10,55.333646,2023-12-31,FY,2023,62.875842,5.177414,0.725056,4.149503,...,4.836007,2.328605,11.020963,0.024685,0.036775,0.002715,0.009628,21.890079,-183.678383,BN
2,BN,4.931355e+10,9.492502,2022-12-31,FY,2022,59.182775,4.616268,3.314195,5.582775,...,4.760070,6.161108,8.620004,0.118735,0.032945,0.013883,0.020040,45.622639,-164.759968,BN
3,BN,7.558044e+10,6.101101,2021-12-31,FY,2021,49.287992,4.478360,8.062480,5.124634,...,4.704373,5.684846,7.479257,0.267242,0.039700,0.036417,0.019488,73.978852,-143.347218,BN
4,BN,5.081327e+10,71.871666,2020-12-31,FY,2020,41.519121,2.654493,0.467778,5.518724,...,6.205083,1.832618,4.573938,0.019728,0.014559,0.002323,0.017964,15.797684,-127.612313,BN


In [15]:
print(len(metrics_df['Stock'].unique()) + len(df_metrics['Stock'].unique()))

352


In [16]:
upd_df = pd.concat([metrics_df, df_metrics], ignore_index=True)

upd_df.tail()

,symbol,market_cap,pe_ratio,period_ending,fiscal_period,calendar_year,revenue_per_share,capex_per_share,net_income_per_share,operating_cash_flow_per_share,...,receivables_turnover,payables_turnover,inventory_turnover,return_on_equity,return_on_invested_capital,return_on_tangible_assets,dividend_yield,graham_number,graham_net_net,Stock
1752,HEI,3.391455e+10,65.967630,2024-10-31,FY,2024,27.862258,0.420794,3.713185,4.856235,...,5.928290,11.872977,2.011995,0.141360,0.096315,0.178649,0.000857,46.846217,-19.210552,HEI
1753,HEI,2.173148e+10,53.844626,2023-10-31,FY,2023,21.635784,0.360346,2.941983,3.271021,...,4.781274,8.813398,1.790128,0.128289,0.076195,0.157442,0.001259,38.961626,-18.177164,HEI
1754,HEI,2.212067e+10,62.900878,2022-10-31,FY,2022,16.236468,0.235144,2.585655,3.439865,...,5.679461,11.544843,2.310094,0.132792,0.107087,0.208123,0.001106,33.656988,-2.920594,HEI
1755,HEI,1.864792e+10,61.297491,2021-10-31,FY,2021,13.786575,0.267377,2.248053,3.281587,...,5.740701,13.306123,2.381046,0.132446,0.103990,0.207559,0.001233,29.300743,-4.510597,HEI
1756,HEI,1.415591e+10,45.084806,2020-10-31,FY,2020,13.261269,0.170236,2.330053,3.036088,...,6.597489,14.492727,2.385298,0.156164,0.104375,0.198034,0.001522,27.968357,-5.161279,HEI


In [17]:
len(upd_df['Stock'].unique())

352

In [18]:
upd_df.shape

(1757, 62)

In [19]:
upd_df.to_csv("data/stocks_metrics.csv", index=False)

## Download Balance, Cash and Income Growth Data

## Download Price Data

In [2]:
ratio_df = pd.read_csv("data/stocks_ratios.csv")

In [3]:
ratio_df['period_ending'] = pd.to_datetime(ratio_df['period_ending'])

In [91]:
len(ratio_df['Stock'].unique())

867

In [68]:
ids = ratio_df['Stock'].unique().tolist()

In [70]:
from pandas.errors import EmptyDataError

def download_historical_prices(ids, start_date="2020-01-01", provider="yfinance"):

    df_list = []

    for id in ids:
        try:
            price = obb.equity.price.historical(symbol=id, start_date=start_date, provider=provider).to_df()
            price['Stock'] = id
            df_list.append(price)
        
        except EmptyDataError:
            print(f"No data found for the symbol {id}")

        except Exception as e:
            print(f"Error with {id}: {e}")


    return pd.concat(df_list, ignore_index=False) if df_list else pd.DataFrame()

In [ ]:
price_df = download_historical_prices(ids, start_date="2020-01-01")
price_df.head()

In [16]:
price_df.to_csv("data/stock_prices.csv", index=True)

In [4]:
price_df = pd.read_csv("data/stock_prices.csv")

In [5]:
price_df['date'] = pd.to_datetime(price_df['date'])

In [81]:
price_df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'split_ratio',
       'dividend', 'Stock'],
      dtype='object')

In [82]:
price_df = price_df[price_df['close'].notna()]
len(price_df['Stock'].unique())

841

In [6]:
def find_closest_price(stock, date): #Find the closest date for the period ending date
    stock_prices = price_df[price_df['Stock'] == stock]  # Filter stock prices
    if stock_prices.empty:
        return (pd.NaT, None, None)  # Return None if no prices found
    closest_row = stock_prices.iloc[(stock_prices['date'] - date).abs().argsort()[:1]]
    return tuple(closest_row[['date', 'close', 'split_ratio']].values[0]) if not closest_row.empty else (pd.NaT, None, None)

matched_prices = ratio_df.apply(lambda row: find_closest_price(row['Stock'], row['period_ending']), axis=1)

ratio_df[['closest_date', 'close_price', 'split_ratio']] = pd.DataFrame(matched_prices.tolist(), index=ratio_df.index)

In [7]:
filtered_df = ratio_df[ratio_df['close_price'].notna()]
len(filtered_df['Stock'].unique())

841

In [8]:
df_2 = ratio_df.copy()

In [9]:
price_df['split_ratio'].fillna(1, inplace=True)

/var/folders/n8/s5gn3hhs4z90dmrnk47r93fh0000gn/T/ipykernel_11200/4039875581.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  price_df['split_ratio'].fillna(1, inplace=True)


In [10]:
import numpy as np

def adjust_price(stock, date, price):
    if pd.isna(price) or pd.isna(date):  # If price or date is missing, return NaN
        return np.nan
    
    stock_splits = price_df[(price_df['Stock'] == stock) & (price_df['date'] >= date)]
    
    if stock_splits.empty:
        return price  # If no splits found, return the original price
    
    split_factor = stock_splits.sort_values(by="date")['split_ratio'].replace(0, 1).cumprod().iloc[-1] if not stock_splits.empty else 1
    
    return price / split_factor if split_factor != 0 else price

ratio_df['adjusted_price'] = ratio_df.apply(lambda row: adjust_price(row['Stock'], row['closest_date'], row['close_price']), axis=1)

In [11]:
filtered_df = ratio_df[ratio_df['adjusted_price'].notna()]
len(filtered_df['Stock'].unique())

841

In [12]:
df_r = ratio_df.copy()

In [13]:
ratio_df = ratio_df.sort_values(by=['Stock', 'period_ending'])

In [14]:
ratio_df['prev_adjusted_price'] = ratio_df.groupby('Stock')['adjusted_price'].shift(1)
ratio_df['return'] = (ratio_df['adjusted_price'] - ratio_df['prev_adjusted_price']) / ratio_df['prev_adjusted_price']

In [15]:
new_column_order = ['Stock', 'return', 'adjusted_price', 'close_price'] + [col for col in ratio_df.columns if col not in ['Stock', 'return', 'adjusted_price', 'close_price']]

ratio_df = ratio_df[new_column_order]

In [16]:
ratio_df.drop(columns=['prev_adjusted_price', 'closest_date','split_ratio'], inplace=True)

In [17]:
ratio_df.head()

,Stock,return,adjusted_price,close_price,period_ending,fiscal_period,fiscal_year,current_ratio,quick_ratio,cash_ratio,...,price_earnings_ratio,price_to_free_cash_flows_ratio,price_to_operating_cash_flows_ratio,price_cash_flow_ratio,price_earnings_to_growth_ratio,price_sales_ratio,dividend_yield,enterprise_value_multiple,price_fair_value,interest_coverage
1846,A,NaN,102.089996,102.089996,2020-10-31,FY,2020,2.327880,1.837082,0.982277,...,43.874562,39.333928,34.251694,34.251694,-1.385299,5.908562,0.007037,23.993947,6.473591,10.846154
1845,A,0.538447,157.059998,157.059998,2021-10-31,FY,2021,2.224239,1.738290,0.868852,...,39.459702,36.841235,32.152350,32.152350,0.557219,7.555980,0.004943,28.400719,8.859944,16.629630
1844,A,-0.119126,138.350006,138.350006,2022-10-31,FY,2022,2.030091,1.472327,0.565825,...,32.987759,40.515818,31.529459,31.529459,6.251966,6.040691,0.006044,21.606143,7.797672,19.261905
1843,A,-0.252837,103.370003,103.370003,2023-10-31,FY,2023,2.611354,1.968185,0.991890,...,24.508694,20.617897,17.150553,17.150553,34.230475,4.447648,0.008720,18.517780,5.199449,14.210526
1842,A,0.260617,130.309998,130.309998,2024-10-31,FY,2024,2.089182,1.576253,0.701319,...,29.317223,27.523598,21.581896,21.581896,5.623576,5.804900,0.007251,26.781519,6.407240,15.500000


In [18]:
stocks = ratio_df[ratio_df['return'].notna()]

In [19]:
len(stocks['Stock'].unique())

841

In [20]:
stocks.shape

(3357, 61)

In [21]:
stocks.to_csv("data/stocks.csv", index=False)

## Download Company Profile Data

In [31]:
stocks = pd.read_csv("data/stocks.csv")

In [32]:
companies = stocks['Stock'].unique().tolist()

In [5]:
from pandas.errors import EmptyDataError

def get_company_profile(companies):

    data = []

    for company in companies:
        try:
            profile = obb.equity.profile(symbol=company, provider='fmp').to_df() 
            profile['Stock'] = company
            data.append(profile)

        except EmptyDataError:
            print(f"No data found for the symbol {company}")

        except Exception as e:
            print(f"Error with {company}: {e}")

    return pd.concat(data, ignore_index=True) if data else pd.DataFrame()

In [6]:
df_profile = pd.read_csv("data/companies_profile.csv")

In [7]:
downloaded_companies = df_profile['Stock'].unique().tolist()

In [8]:
def get_remaining_companies(companies, downloaded_companies, limit=250):
    return [company for company in companies if company not in downloaded_companies][:limit]

In [9]:
companies = get_remaining_companies(companies, downloaded_companies)

In [19]:
companies[:5]

['TSN', 'TT', 'TTD', 'TTE', 'TTWO']

In [11]:
df = get_company_profile(companies)

In [12]:
df_profile = pd.concat([df_profile, df], ignore_index=True)

In [13]:
df_profile.tail()

,symbol,name,cik,cusip,isin,stock_exchange,long_description,ceo,company_url,business_phone_no,...,image,currency,market_cap,last_price,year_high,year_low,volume_avg,annualized_dividend_amount,beta,Stock
836,ZG,"Zillow Group, Inc. Class A",0001617640,98954M101,US98954M1018,NASDAQ Global Select,"Zillow Group, Inc., a digital real estate comp...",Mr. Jeremy Wacksman,https://www.zillowgroup.com,206 470 7000,...,https://images.financialmodelingprep.com/symbo...,USD,16589768563,67.20,86.58,38.06,609268,0.000,2.337,ZG
837,ZM,"Zoom Video Communications, Inc.",0001585521,98980L101,US98980L1017,NASDAQ Global Select,"Zoom Video Communications, Inc. provides unifi...",Mr. Eric S. Yuan,https://www.zoom.us,888 799 9666,...,https://images.financialmodelingprep.com/symbo...,USD,22843702008,74.84,92.80,55.06,2606025,0.000,0.253,ZM
838,ZS,"Zscaler, Inc.",0001713683,98980G102,US98980G1022,NASDAQ Global Select,"Zscaler, Inc. operates as a cloud security com...",Mr. Jagtar Singh Chaudhry,https://www.zscaler.com,408 533 0288,...,https://images.financialmodelingprep.com/symbo...,USD,31783354460,207.14,217.84,153.45,1813683,0.000,0.878,ZS
839,ZTO,ZTO Express (Cayman) Inc.,0001677250,98980A105,US98980A1051,New York Stock Exchange,ZTO Express (Cayman) Inc. provides express del...,Mr. Meisong Lai,https://www.zto.com,86 21 5980 4508,...,https://images.financialmodelingprep.com/symbo...,USD,11736798261,19.82,27.50,17.89,2643056,0.970,-0.126,ZTO
840,ZTS,Zoetis Inc.,0001555280,98978V103,US98978V1035,New York Stock Exchange,"Zoetis Inc. discovers, develops, manufactures,...",Ms. Kristin C. Peck,https://www.zoetis.com,973 822 7000,...,https://images.financialmodelingprep.com/symbo...,USD,73039353120,163.11,200.33,144.80,2984611,1.796,0.941,ZTS


In [14]:
len(df_profile['symbol'].unique())

841

In [15]:
len(stocks['Stock'].unique())

841

In [16]:
df_profile.to_csv("data/companies_profile.csv", index=False)

In [38]:
df_profile['sector'].unique

<bound method Series.unique of 0                  Healthcare
1                  Technology
2                  Healthcare
3          Consumer Defensive
4           Consumer Cyclical
                ...          
836    Communication Services
837                Technology
838                Technology
839               Industrials
840                Healthcare
Name: sector, Length: 841, dtype: object>